<a href="https://colab.research.google.com/github/joansaldanha21/DSBA/blob/master/Datacleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',

'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [np.nan, np.nan, np.nan, 10075, np.nan],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})




In [247]:
print (df.dtypes)

From_To          object
FlightNumber    float64
RecentDelays     object
Airline          object
dtype: object


In [38]:
# 1 Some values in the the FlightNumber column are missing. These numbers are
# meant to increase by 10 with each row so 10055 and 10075 need to be put in
# place. Fill in these missing numbers and make the column an integer column
# (instead of a float column).
# This function will look for a value in the col and build the remaining values by the increment given
def Missing_Val_Increment_by(Colname,Incrementby):
  if (df[Colname].isna().sum()) != len(df[Colname]):
    Col_Loc = df.columns.get_loc(Colname)
    for i in df[Colname].index:
        if (pd.isnull(df.iloc[i,Col_Loc])):
          a = i+1
          if (a >= len(df[Colname]) and i > 0):
              a=0
          while (pd.isnull(df.iloc[a,Col_Loc]) and a != i):
            a=a+1
            if (a >= len(df[Colname]) and i > 0):
             a=0
            else:
             a=a
          if a > i:
             df.iloc[i,Col_Loc] = df.iloc[a,Col_Loc]-(Incrementby*(a-i)) 
          else:
            df.iloc[i,Col_Loc] = df.iloc[a,Col_Loc]+(Incrementby*(i+a))
  else:
    print("All records are null")
  
Missing_Val_Increment_by("FlightNumber",10)
print(df)



            From_To  FlightNumber  RecentDelays              Airline
0      LoNDon_paris       10045.0      [23, 47]               KLM(!)
1      MAdrid_miLAN       10055.0            []    <Air France> (12)
2  londON_StockhOlm       10065.0  [24, 43, 87]  (British Airways. )
3    Budapest_PaRis       10075.0          [13]       12. Air France
4   Brussels_londOn       10085.0      [67, 32]          "Swiss Air"


In [39]:
# 2 The From_To column would be better as two separate columns! Split each
# string on the underscore delimiter _ to give a new temporary DataFrame with
# the correct values. Assign the correct column names to this temporary
# DataFrame.
tmp = df.loc[:]
tmp[['From','To']] = tmp.From_To.str.split("_",expand=True)
#tmp.drop(tmp[['From_To']],axis=1,inplace=True)
print(tmp)


            From_To  FlightNumber  ...      From         To
0      LoNDon_paris       10045.0  ...    LoNDon      paris
1      MAdrid_miLAN       10055.0  ...    MAdrid      miLAN
2  londON_StockhOlm       10065.0  ...    londON  StockhOlm
3    Budapest_PaRis       10075.0  ...  Budapest      PaRis
4   Brussels_londOn       10085.0  ...  Brussels     londOn

[5 rows x 6 columns]


In [40]:
# 3 Notice how the capitalisation of the city names is all mixed up in this
# temporary DataFrame. Standardise the strings so that only the first letter is
# uppercase (e.g. "londON" should become "London".)

tmp1 = tmp[:]

def to_camelcase(str):
  return ' '.join([t.title() for t in str.split()])

Col_Loc = tmp1.columns.get_loc('From')
print(Col_Loc)
for i in tmp1['From'].index:
  print(tmp1.iloc[i,Col_Loc])
  tmp1.iloc[i,Col_Loc]=(to_camelcase(tmp1.iloc[i,Col_Loc]))

print(tmp1)

4
LoNDon
MAdrid
londON
Budapest
Brussels
            From_To  FlightNumber  ...      From         To
0      LoNDon_paris       10045.0  ...    London      paris
1      MAdrid_miLAN       10055.0  ...    Madrid      miLAN
2  londON_StockhOlm       10065.0  ...    London  StockhOlm
3    Budapest_PaRis       10075.0  ...  Budapest      PaRis
4   Brussels_londOn       10085.0  ...  Brussels     londOn

[5 rows x 6 columns]


In [41]:
# 4 Delete the From_To column from df and attach the temporary DataFrame
# from the previous questions.
df.drop(df[['From_To']],axis=1,inplace=True)
df[['From','To']]=tmp1[['From','To']]
print(df)


   FlightNumber  RecentDelays              Airline      From         To
0       10045.0      [23, 47]               KLM(!)    London      paris
1       10055.0            []    <Air France> (12)    Madrid      miLAN
2       10065.0  [24, 43, 87]  (British Airways. )    London  StockhOlm
3       10075.0          [13]       12. Air France  Budapest      PaRis
4       10085.0      [67, 32]          "Swiss Air"  Brussels     londOn


In [16]:
# 5 Expand the Series of lists into a DataFrame named delays, rename the columns
# delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
# with delays.
#import itertools
#L=df['RecentDelays']
#print(list(itertools.chain.from_iterable(L)))

Rdelays = df['RecentDelays'].apply(pd.Series)
Rdelays = Rdelays.rename(columns = lambda x : 'RecentDelays_' + str(x))
Rdelays = pd.concat([df[:], Rdelays[:]], axis=1)
Rdelays = Rdelays.drop(Rdelays[['RecentDelays']],axis=1)
Rdelays

,From_To,FlightNumber,Airline,RecentDelays_0,RecentDelays_1,RecentDelays_2
0,LoNDon_paris,NaN,KLM(!),23.0,47.0,NaN
1,MAdrid_miLAN,NaN,<Air France> (12),NaN,NaN,NaN
2,londON_StockhOlm,NaN,(British Airways. ),24.0,43.0,87.0
3,Budapest_PaRis,10075.0,12. Air France,13.0,NaN,NaN
4,Brussels_londOn,NaN,"""Swiss Air""",67.0,32.0,NaN
